#### 2023-05-11

##### 1교시

In [1]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download
('wordnet')
import nltk
nltk.download
('omw-1.4')

lemma = WordNetLemmatizer()
print(lemma.lemmatize('happiest','a'),lemma.lemmatize('swam','v'),lemma.lemmatize('apples','n'))
print(lemma.lemmatize('fallen','v'),lemma.lemmatize('busier','a'),lemma.lemmatize('cries','v')) 

happy swim apple
fall busy cry


In [9]:
words = {'happiest':'a','swam':'v','apples':'n','fallen':'v','busier':'a','cries':'v'}
for i,k in words.items():
    print(lemma.lemmatize(i,k))

happy
swim
apple
fall
busy
cry


In [ ]:
# pipeline 내에 벡터화 도구들을 나열할 때, 어떤 자료구조여야 하는가?
리스트 내 튜플, 튜플의 요소는 (객체명, 클래스명(하이퍼 파라미터))

In [ ]:
# 유사도를 측정하는 방법은?
코사인 유사도

In [20]:
import numpy as np
# a = (3,4), b = (4,1)
a = np.array((3,4))
b = np.array((4,1))
c = a@b
d = np.sqrt(sum(np.square(b)))*np.sqrt(sum(np.square(a)))
c/d

0.7761140001162655

In [ ]:
# TF-IDF식을 한글로 서술하시오
TF * IDF
TF : Term Frequency
IDF : inverse DF = log(총 문서개수 / 특정 단어가 나타난 문서의 개수+1)
DF : Document Frequency (그 단어가 나타난 문서의 개수 / 총문서 개수)

##### 2교시

In [ ]:
pos : 품사
semantic : 문맥
영어 -> 단어쪼개기 -> stemmoing -> 숫자 -> 빈도세기 -> 학습...

In [ ]:
KoNLPy 형태소 : 단어로서 의미를 가지는 최소 단위 

In [1]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import konlpy

In [31]:
import os 
os.getcwd()
os.chdir('/Users/sonsoohan/Downloads/수업자료/ml-latest-small')

In [2]:
import pandas as pd
train_df = pd.read_csv('ratings_train.txt',sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [3]:
test_df = pd.read_csv('ratings_test.txt',sep='\t')
test_df.head(3)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0


In [4]:
train_df = train_df.fillna('')
test_df = test_df.fillna('')

In [5]:
import re
train_df['document'] = train_df['document'].apply(lambda x: re.sub(r'\d+','',x))
test_df['document'] = test_df['document'].apply(lambda x: re.sub(r'\d+','',x))

In [6]:
# train_df의 10%만 뽑기
train_df = train_df.sample(frac=0.1)
test_df = train_df.sample(frac=0.1)

##### 3교시

In [8]:
from konlpy.tag import Twitter

twitter = Twitter()
def tw_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Twitter 객체의 morphs( ) 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2) 
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

/Users/sonsoohan/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [10]:
# Logistic Regression 을 이용하여 감성 분석 Classification 수행. 
lg_clf = LogisticRegression(random_state=0, solver='liblinear')

# Parameter C 최적화를 위해 GridSearchCV 를 이용. 
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lg_clf , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )
grid_cv.fit(tfidf_matrix_train , train_df['label'] )
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 3.5} 0.8077


In [11]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함. 
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier 는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ',accuracy_score(test_df['label'],preds))

Logistic Regression 정확도:  0.9493333333333334


##### 5교시

##### 6교시

In [13]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 14.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-macosx_10_9_x86_64.whl size=1122634 sha256=e2c48c0f019397ef392a1d136813a758494030eb7120254305dfef83d2d23779
  Stored in directory: /Users/sonsoohan/Library/Caches/pip/wheels/df/e4/a6/7ad72453dd693f420b0c639bedeec34641738d11b55d8d9b84
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [15]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [16]:
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 Y


Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/sonsoohan/.surprise_data/ml-100k


In [17]:
data = Dataset.load_builtin('ml-100k')

In [18]:
trainset, testset = train_test_split(data, test_size=25, random_state=0)

In [19]:
algo = SVD(random_state=0)
algo.fit(trainset)

In [20]:
predictions = algo.test(testset)
print('prediction type:', type(predictions),'size:',len(predictions))
print('prediction 결과의 최초5개 추출')
predictions[:5]

prediction type: <class 'list'> size: 25
prediction 결과의 최초5개 추출


[Prediction(uid='917', iid='50', r_ui=3.0, est=4.149217910133892, details={'was_impossible': False}),
 Prediction(uid='235', iid='318', r_ui=5.0, est=4.333029650171128, details={'was_impossible': False}),
 Prediction(uid='339', iid='12', r_ui=5.0, est=4.652894591303144, details={'was_impossible': False}),
 Prediction(uid='878', iid='755', r_ui=2.0, est=2.7545011408291202, details={'was_impossible': False}),
 Prediction(uid='618', iid='56', r_ui=4.0, est=3.0213838231732826, details={'was_impossible': False})]

In [21]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.37   {'was_impossible': False}


In [22]:
accuracy.rmse(predictions)

RMSE: 0.9277


0.9277328494227116

In [34]:
ratings = pd.read_csv('ratings.csv')
ratings.to_csv('ratings_noh.csv',index=False,header=False)

In [35]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data = Dataset.load_from_file('ratings_noh.csv',reader=reader)

In [36]:
trainset, testset = train_test_split(data, test_size=25, random_state=0)
algo = SVD(n_factors=50, random_state=0)

In [37]:
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.7940


0.7939608260279408